In [30]:
## https://realpython.com/python-boto3-aws-s3/ ##

In [1]:
import boto3 
ACCESS_ID = 'xxx' 
ACCESS_KEY = 'xxx'
s3_resource = boto3.resource('s3', aws_access_key_id=ACCESS_ID,
                             aws_secret_access_key= ACCESS_KEY)

In [2]:
#Generate unique bucket name

import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

In [5]:
first_bucket_name = s3_resource.create_bucket(Bucket=create_bucket_name('firstpythonbucket'), 
                                                              CreateBucketConfiguration={'LocationConstraint': 'ap-south-1'})

In [12]:
first_bucket_name = first_bucket_name.name 

In [32]:
#Creating files 

def create_temp_file(size, file_name, file_content):
    random_file_name = ''.join([str(uuid.uuid4().hex[:6]), file_name])
    with open(random_file_name, 'w') as f:
        f.write(str(file_content) * size)
    return random_file_name

In [14]:
first_file_name = create_temp_file(300, 'firstfile.txt', 'f')   

In [21]:
first_bucket = s3_resource.Bucket(name=first_bucket_name)
first_object = s3_resource.Object(
    bucket_name=first_bucket_name, key=first_file_name)

In [22]:
print(first_bucket)
print(first_object)

s3.Bucket(name='firstpythonbucket9d764cd5-34ca-42b3-b849-ab7bb2accb51')
s3.Object(bucket_name='firstpythonbucket9d764cd5-34ca-42b3-b849-ab7bb2accb51', key='c62c12firstfile.txt')


In [23]:
#Creating similar sub-resources
first_bucket_again = first_object.Bucket()
first_object_again = first_bucket.Object(first_file_name)

In [24]:
print(first_bucket_again)
print(first_object_again)

s3.Bucket(name='firstpythonbucket9d764cd5-34ca-42b3-b849-ab7bb2accb51')
s3.Object(bucket_name='firstpythonbucket9d764cd5-34ca-42b3-b849-ab7bb2accb51', key='c62c12firstfile.txt')


In [33]:
#Uploading files
first_object.upload_file(first_file_name)

In [34]:
first_bucket.upload_file(
    Filename=first_file_name, Key=first_file_name)

In [39]:
#downloadng a file
s3_resource.Object(first_bucket_name, first_file_name).download_file(
    f'download/{first_file_name}')

In [40]:
#Copying from one bucket to other
def copy_to_bucket(bucket_from_name, bucket_to_name, file_name):
    copy_source = {
        'Bucket': bucket_from_name,
        'Key': file_name
    }
    s3_resource.Object(bucket_to_name, file_name).copy(copy_source)

copy_to_bucket(first_bucket_name, second_bucket_name, first_file_name)

In [ ]:
#Deleting object
s3_resource.Object(second_bucket_name, first_file_name).delete()


In [41]:
#ACL (Access Control List)
second_file_name = create_temp_file(400, 'secondfile.txt', 's')
second_object = s3_resource.Object(first_bucket.name, second_file_name)
second_object.upload_file(second_file_name, ExtraArgs={
                          'ACL': 'public-read'})

In [ ]:
#making object private
response = second_object_acl.put(ACL='private')
second_object_acl.grants

In [43]:
#Upload with Encryption

third_file_name = create_temp_file(300, 'thirdfile.txt', 't')
third_object = s3_resource.Object(first_bucket_name, third_file_name)
third_object.upload_file(third_file_name, ExtraArgs={
                         'ServerSideEncryption': 'AES256'})

In [44]:
third_object.server_side_encryption

'AES256'

In [47]:
#Versoning

def enable_bucket_versioning(bucket_name):
    bkt_versioning = s3_resource.BucketVersioning(bucket_name)
    bkt_versioning.enable()
    print(bkt_versioning.status)

In [48]:
enable_bucket_versioning(first_bucket_name)

Enabled


In [49]:
s3_resource.Object(first_bucket_name, first_file_name).upload_file(
   first_file_name)
s3_resource.Object(first_bucket_name, first_file_name).upload_file(
   third_file_name)

In [50]:
s3_resource.Object(first_bucket_name, second_file_name).upload_file(
    second_file_name)

In [51]:
s3_resource.Object(first_bucket_name, first_file_name).version_id

'3eGR5UJQk962WhfRc0prR5EKnbYjo.4n'

In [52]:
#bucket traversal
for bucket in s3_resource.buckets.all():
    print(bucket.name)

firstpythonbucket9d764cd5-34ca-42b3-b849-ab7bb2accb51


In [53]:
#object traversal
for obj in first_bucket.objects.all():
    print(obj.key)

1b177esecondfile.txt
c62c12firstfile.txt
f04694thirdfile.txt


In [54]:
#Object Summary

for obj in first_bucket.objects.all():
    subsrc = obj.Object()
    print(obj.key, obj.storage_class, obj.last_modified,
          subsrc.version_id, subsrc.metadata)

1b177esecondfile.txt STANDARD 2019-07-09 06:41:16+00:00 bwDMbrN86BoQR3B6QuC2DfwIGDFVN3sa {}
c62c12firstfile.txt STANDARD 2019-07-09 06:40:28+00:00 3eGR5UJQk962WhfRc0prR5EKnbYjo.4n {}
f04694thirdfile.txt STANDARD 2019-07-09 06:35:42+00:00 null {}


In [55]:
#deleting Non-empty buckets

def delete_all_objects(bucket_name):
    res = []
    bucket=s3_resource.Bucket(bucket_name)
    for obj_version in bucket.object_versions.all():
        res.append({'Key': obj_version.object_key,
                    'VersionId': obj_version.id})
    print(res)
    bucket.delete_objects(Delete={'Objects': res})
delete_all_objects(first_bucket_name)

In [ ]:
#Deleting Bucket

s3_resource.Bucket(first_bucket_name).delete()